In [2]:
import pandas as pd
import requests
import gzip
import shutil
from pathlib import Path
import csv

In [14]:
df = pd.read_csv('../data/processed/openfoodfacts_processed_10_000.csv')

In [15]:
df.nunique()

code                                                     7736
url                                                      7736
product_name                                             7123
quantity                                                 1651
brands                                                   2435
brands_tags                                              2072
categories                                               5586
categories_tags                                          4894
categories_en                                            4894
labels                                                   5732
labels_tags                                              5246
labels_en                                                5246
countries                                                 958
countries_tags                                            614
countries_en                                              614
ingredients_text                                         7481
ingredie

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7793 entries, 0 to 7792
Data columns (total 49 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   code                                                   7793 non-null   int64  
 1   url                                                    7793 non-null   object 
 2   product_name                                           7793 non-null   object 
 3   quantity                                               7793 non-null   object 
 4   brands                                                 7793 non-null   object 
 5   brands_tags                                            7793 non-null   object 
 6   categories                                             7793 non-null   object 
 7   categories_tags                                        7793 non-null   object 
 8   categories_en                                   

In [29]:
df[['nova_group']].value_counts().sort_index()

nova_group
1              433
2               93
3             1391
4             5876
Name: count, dtype: int64

In [19]:
df[['nutriscore_grade']].value_counts().sort_index()

nutriscore_grade
a                    799
b                    929
c                   2392
d                   2038
e                   1635
Name: count, dtype: int64

In [20]:
df[['nutriscore_score']].value_counts().sort_index()

nutriscore_score
-16                  20
-15                   3
-13                   4
-11                  10
-10                  15
-9                   14
-8                   25
-7                   16
-6                   27
-5                   45
-4                   64
-3                   88
-2                  143
-1                  209
 0                  280
 1                  318
 2                  447
 3                  429
 4                  453
 5                  428
 6                  326
 7                  291
 8                  240
 9                  167
 10                 175
 11                 318
 12                 262
 13                 265
 14                 303
 15                 228
 16                 215
 17                 202
 18                 208
 19                 224
 20                 179
 21                 161
 22                 147
 23                  99
 24                  95
 25                  93
 26                 100

In [23]:
df[['product_name','nutriscore_grade', 'nutriscore_score']].head(100)

,product_name,nutriscore_grade,nutriscore_score
0,Barre daily carotte,a,-1
1,Cold Brew Coffee,e,10
2,this is food happy banana,e,10
3,yfood Crazy Coconut,e,10
4,Hot & Savoury - Tomato & Herb,c,10
...,...,...,...
95,chocapic,c,7
96,Renutryl Booster saveur Chocolat,d,9
97,Viva chocolat 25cl,d,7
98,NESTLE LION Céréales 750g,d,14


In [30]:
df['categories_tags'].unique()

array(['en:plant-based-foods-and-beverages,en:plant-based-foods,en:snacks,en:sweet-snacks,en:bars,en:dietary-supplements,en:bodybuilding-supplements,en:protein-bars',
       'en:beverages-and-beverages-preparations,en:beverages',
       'en:beverages-and-beverages-preparations,en:beverages,en:meals,de:trinkmahlzeit,en:drink-meal',
       ...,
       'en:plant-based-foods-and-beverages,en:beverages,en:plant-based-foods,en:fruits-and-vegetables-based-foods,en:legumes-and-their-products,en:dairy-substitutes,en:vegetables-based-foods,en:milk-substitutes,en:plant-based-beverages,en:plant-based-milk-alternatives,en:legume-based-drinks,en:soy-based-drinks',
       'en:dairies,en:fermented-foods,en:fermented-milk-products,en:cheeses,en:cow-cheeses,en:french-cheeses,en:pasteurized-cheeses,fr:fromages-a-pate-molle',
       'en:snacks,en:sweet-snacks,en:confectioneries,en:fruit-confectioneries,en:candies,en:liquorice-candies'],
      shape=(4894,), dtype=object)

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df.head()

,code,url,product_name,quantity,brands,brands_tags,categories,categories_tags,categories_en,labels,labels_tags,labels_en,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,additives_n,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,nutrient_levels_tags,product_quantity,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,nutrition-score-fr_100g,nutriscore_score,nutriscore_grade,nova_group
0,3701299421039,http://world-en.openfoodfacts.org/product/3701...,Barre daily carotte,51 g,Vitaline,vitaline,"Aliments et boissons à base de végétaux, Alime...","en:plant-based-foods-and-beverages,en:plant-ba...","Plant-based foods and beverages,Plant-based fo...","Sans gluten, Bio, Végétarien, Bio européen, Vé...","en:no-gluten,en:organic,en:vegetarian,en:eu-or...","No gluten,Organic,Vegetarian,EU Organic,Vegan,...",France,en:france,France,"Céréales, noix et graines (46% : flocons d'avo...","en:cereal,en:walnut,en:nut,en:tree-nut,en:seed...","en:palm-oil-free,en:vegan,en:vegetarian",1,en:biscuits-and-cakes,"en:sugary-snacks,en:biscuits-and-cakes","Sugary snacks,Biscuits and cakes","en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...","en:fat-in-moderate-quantity,en:saturated-fat-i...",51.0,en:protein-bars,Protein bars,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,393.0,1642.0,14.3,2.0,43.7,10.6,13.3,0.30,0.120,72.80000,-1,-1,a,4
1,4260556630045,http://world-en.openfoodfacts.org/product/4260...,Cold Brew Coffee,500 ml,yfood,yfood,Getränke,"en:beverages-and-beverages-preparations,en:bev...","Beverages and beverages preparations,Beverages","Glutenfrei, Natürliche Aromen, Ohne Zuckerzusa...","en:no-gluten,en:natural-flavors,en:no-added-su...","No gluten,Natural flavors,No added sugar,No la...","Germany, en:switzerland","en:germany,en:switzerland","Germany,Switzerland","Fettarme _Milch_, Wasser, _Milch_eiweiß, Pflan...","en:skimmed-milk,en:dairy,en:milk,en:water,de:m...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...",4,en:sweetened-beverages,"en:beverages,en:sweetened-beverages","Beverages,Sweetened beverages","en:to-be-checked, en:complete, en:nutrition-fa...","en:to-be-checked,en:complete,en:nutrition-fact...","To be checked,Complete,Nutrition facts complet...","en:fat-in-moderate-quantity,en:saturated-fat-i...",500.0,en:beverages,Beverages,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,100.0,419.0,4.6,1.0,6.9,4.5,6.8,0.17,0.068,4.52500,10,10,e,4
2,4260556630380,http://world-en.openfoodfacts.org/product/4260...,this is food happy banana,1pcs,"yfood,yF","yfood,yf",Getränke,"en:beverages-and-beverages-preparations,en:bev...","Beverages and beverages preparations,Beverages","Glutenfrei,Vegetarisch,Ohne Zuckerzusatz,Lakto...","en:no-gluten,en:vegetarian,en:no-added-sugar,e...","No gluten,Vegetarian,No added sugar,No lactose...",Deutschland,en:germany,Germany,"Fettarme Milch, Wasser, Milcheiweiß, pflanzlic...","en:skimmed-milk,en:dairy,en:milk,en:water,en:m...","en:palm-oil-free,en:non-vegan,en:vegetarian",4,en:sweetened-beverages,"en:beverages,en:sweetened-beverages","Beverages,Sweetened beverages","en:

In [35]:
df[['product_name','nutriscore_grade', 'nutriscore_score','nova_group']].head(100)

,product_name,nutriscore_grade,nutriscore_score,nova_group
0,Barre daily carotte,a,-1,4
1,Cold Brew Coffee,e,10,4
2,this is food happy banana,e,10,4
3,yfood Crazy Coconut,e,10,4
4,Hot & Savoury - Tomato & Herb,c,10,4
5,Hot & Savoury - Thai Green Curry,c,9,4
6,Cool Mint Chocolate Clif Energy Bar,d,12,4
7,Fresh Berry,e,10,4
8,Crème saveur vanille,d,15,4
9,Boulgour Bio,a,-7,1


In [ ]:
from y_new import categorize_nutrition

In [39]:
df_new = categorize_nutrition(df)

In [43]:
df_new[['product_name','nutriscore_grade', 'nutriscore_score','nova_group', 'y','nutrition_category']].head(20)

,product_name,nutriscore_grade,nutriscore_score,nova_group,y,nutrition_category
0,Barre daily carotte,a,-1,4,0.600,Poor
1,Cold Brew Coffee,e,10,4,1.000,Bad
2,this is food happy banana,e,10,4,1.000,Bad
3,yfood Crazy Coconut,e,10,4,1.000,Bad
4,Hot & Savoury - Tomato & Herb,c,10,4,0.800,Bad
5,Hot & Savoury - Thai Green Curry,c,9,4,0.800,Bad
6,Cool Mint Chocolate Clif Energy Bar,d,12,4,0.900,Bad
7,Fresh Berry,e,10,4,1.000,Bad
8,Crème saveur vanille,d,15,4,0.900,Bad
9,Boulgour Bio,a,-7,1,0.225,Excellent
